# Working code for translating openpose to st-gcn format 

In [3]:
%%capture
# Kjører openpose. Skriver openpose-json-filer til path_store_file

path_openpose_exe = "./build/examples/openpose/openpose.bin"
path_videos = "videos/"
video_name = "Pullups_cut.mp4"
full_path_videos = path_videos + video_name
path_store_file = "/home/stian/openpose/people_pose_data/openpose_format/"
! $path_openpose_exe --video $full_path_videos --write_json $path_store_file --model_pose COCO --keypoint_scale 3

In [4]:
# Change output filenames from openpose 
# Necessary because of p.glob() in st-gcn "translator"
import os
from natsort import natsorted
home_path = '/home/stian/openpose/'
file_path = 'people_pose_data/openpose_format/'
full_path = home_path + file_path
counter = 0
file_list = natsorted(os.listdir(file_path))

for file in file_list:
    new_filename = str(counter) + '_keypoints.json'
    print("\nNew file name: " + new_filename)
    print("Old file name: " + file)
    os.rename(os.path.join(full_path, file), os.path.join(full_path, new_filename))
    counter += 1


New file name: 0_keypoints.json
Old file name: Pullups_cut_000000000000_keypoints.json

New file name: 1_keypoints.json
Old file name: Pullups_cut_000000000001_keypoints.json

New file name: 2_keypoints.json
Old file name: Pullups_cut_000000000002_keypoints.json

New file name: 3_keypoints.json
Old file name: Pullups_cut_000000000003_keypoints.json

New file name: 4_keypoints.json
Old file name: Pullups_cut_000000000004_keypoints.json

New file name: 5_keypoints.json
Old file name: Pullups_cut_000000000005_keypoints.json

New file name: 6_keypoints.json
Old file name: Pullups_cut_000000000006_keypoints.json

New file name: 7_keypoints.json
Old file name: Pullups_cut_000000000007_keypoints.json

New file name: 8_keypoints.json
Old file name: Pullups_cut_000000000008_keypoints.json

New file name: 9_keypoints.json
Old file name: Pullups_cut_000000000009_keypoints.json

New file name: 10_keypoints.json
Old file name: Pullups_cut_000000000010_keypoints.json

New file name: 11_keypoints.js

In [6]:
import os
from pathlib import Path
import json
from natsort import natsorted
import sklearn

def open_pose_to_stgcn():
    p = 'people_pose_data/openpose_format/' #Path til json-filene på OpenPose format (som legges til av OpenPose)
    list_data_file = 'data_list(copy).dat' #Antall frames en ønsker? Tekstfil med bare tall. 
    labels = {}
    with open(list_data_file, 'r') as f:
        for line in f: 
            vid  = int(line) # Henter ut frame nr (skal egentlig være film nr. --> Når vi får kombinert Openpose filene). 
            #vid = 0 --> prøver å kombinere alt til èn fil. 
            j = 0       
            stgcn_data_array = []
            stgcn_data = {}
            dest_path = 'people_pose_data/st-gcn_format/' + 'lord_of_the_files' + '.json' #Lagrer st-gcn formatet her
            file_list =  natsorted(os.listdir(p))
            for path in file_list: # each json file (rekursivt kall), men filnavnet må begynne på frame nr.
                print('path = ' + str(path))
                print('dest_path = ' + str(dest_path))
                json_path = p + path
                j += 1
                frame_id = int(((json_path.split('/')[-1]).split('.')[0]).split('_')[0]) 
                print("frame_id: " + str(frame_id))
                frame_data = {'frame_index': frame_id}
                data = json.load(open(json_path))
                skeletons = []
                for person in data['people']:
                    score, coordinates = [], []
                    skeleton = {}
                    keypoints = person['pose_keypoints_2d']
                    for i in range(0,len(keypoints),3):
                        coordinates +=  [keypoints[i], keypoints[i + 1]]
                        score += [keypoints[i + 2]]
                    #c_min, c_max = min(coordinates), max(coordinates)
                    #s_min, s_max = min(score), max(score)
                    #for i, val in enumerate(coordinates):
                    #    coordinates[i] = (val-c_min) / (c_max-c_min)
                    #for i, val in enumerate(score):
                        #score[i] = (val-s_min) / (s_max-s_min)
                    skeleton['pose'] = coordinates
                    skeleton['score'] = score
                    skeletons += [skeleton]
                frame_data['skeleton'] = skeletons
                stgcn_data_array += [frame_data]
                
            #if j==300: # for å ikke gå over 300 frames - så ved frame nr. 300 - stopp
        labels[str(vid)] = {"has_skeleton": True, 
                "label": "fake_label", 
                "label_index": 0}
        stgcn_data['data'] = stgcn_data_array
        stgcn_data['label'] = 'fake_label'
        stgcn_data['label_index'] = 0
        with open(dest_path, 'w') as outfile:
            json.dump(stgcn_data, outfile)

    labels_file = 'people_pose_data/labels/fake_labels.json' # Path til? #####################
    with open(labels_file, 'w') as label_file:
        json.dump(labels, label_file)

In [7]:
open_pose_to_stgcn()

path = 0_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 0
path = 1_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 1
path = 2_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 2
path = 3_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 3
path = 4_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 4
path = 5_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 5
path = 6_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 6
path = 7_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 7
path = 8_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_files.json
frame_id: 8
path = 9_keypoints.json
dest_path = people_pose_data/st-gcn_format/lord_of_the_fil

In [48]:
def format_json_to_texteditor():
    """ Only for visualiazing """
    #path = 'people_pose_data/st-gcn_format/lord_of_the_files.json'
    path = "/home/stian/st-gcn/data/openpose_estimation/data/Pullups.json"
    ! echo | python -m json.tool $path >> people_pose_data/formated_json/myfile_st-gcn10.txt

In [49]:
format_json_to_texteditor()

# Skriv ut .json fil på en fin måte i teksteditor

In [21]:
# Hvordan st-gcn filen faktisk skal se ut
path_st_gcn_blueprint = '../Downloads/kinetics-skeleton/kinetics_train/--3ouPhoy2A.json'
! echo | python -m json.tool $path_st_gcn_blueprint >> people_pose_data/formated_json/myfile_st-gcn_blueprint_format_3ouPhoy2A.txt

In [22]:
path = 'people_pose_data/openpose_format/Eating Real Spaghetti---3ouPhoy2A_resized_cut_000000000000_keypoints.json'
! echo | python -m json.tool $path >> people_pose_data/formated_json/myfile_openpose_format0.txt

/usr/local/lib/python2.7/json/tool.py [infile [outfile]]


In [37]:
path = 'people_pose_data/st-gcn_format/lord_of_the_files.json'
! echo | python -m json.tool $path >> people_pose_data/formated_json/myfile_st-gcn6.txt

In [24]:
path2 = '/home/stian/st-gcn/data/openpose_estimation/data/Pullups.json'
! echo | python -m json.tool $path2 >> people_pose_data/formated_json/myfile_st-gcn_pullups.txt


In [3]:
import numpy as np
np_file = np.load("/home/stian/st-gcn/data/Kinetics/kinetics-skeleton/train_data.npy")

In [4]:
import pickle
val = pickle.load( open("/home/stian/st-gcn/data/Kinetics/kinetics-skeleton/train_label.pkl", "rb" ))
print(val)

(['QUuC4vJs.json'], [255])


In [5]:
np_file_stgcn = np.load("/home/stian/st-gcn/data/Kinetics/kinetics-skeleton_stgcn/train_data.npy")

In [6]:
print("np-file: " + str(np_file[0]))


np-file: [[[[  0.         0.      ]
   [202.15       0.      ]
   [191.034      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [201.441      0.      ]]

  [[  0.         0.      ]
   [202.155      0.      ]
   [191.042      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [201.439      0.      ]]

  [[  0.         0.      ]
   [199.364      0.      ]
   [186.164      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  ...

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.      

In [7]:
print("np-file_stgcn: " + str(np_file_stgcn[0]))

np-file_stgcn: [[[[  0.         0.      ]
   [202.15       0.      ]
   [191.034      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [201.441      0.      ]]

  [[  0.         0.      ]
   [202.155      0.      ]
   [191.042      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [201.439      0.      ]]

  [[  0.         0.      ]
   [199.364      0.      ]
   [186.164      0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  ...

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]]

  [[  0.         0.      ]
   [  0.         0.      ]
   [  0.         0.      ]
   ...
   [  0.         0.      ]
   [  0.

In [8]:
import json
with open('/home/stian/openpose/people_pose_data/st-gcn_format/kinetics_train/QUuC4vJs.json') as json_file: 
    data = json.load(json_file)
data

{'data': [{'frame_index': 185,
   'skeleton': [{'pose': [0.0,
      0.0,
      0.5183852941176471,
      0.52834765625,
      0.5633411764705882,
      0.531015625,
      0.0,
      0.0,
      0.0,
      0.0,
      0.4713823529411765,
      0.52837109375,
      0.4509941176470588,
      0.33304453125,
      0.5000294117647058,
      0.2353328125,
      0.506120588235294,
      0.75633984375,
      0.0,
      0.0,
      0.0,
      0.0,
      0.45913235294117644,
      0.76987890625,
      0.4346264705882353,
      0.99242578125,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0],
     'score': [0,
      0.519895,
      0.322505,
      0,
      0,
      0.314687,
      0.422202,
      0.388879,
      0.179317,
      0,
      0,
      0.209005,
      0.0680371,
      0,
      0,
      0,
      0,
      0]},
    {'pose': [0.0,
      0.0,
      0.0,
      0.0,
      0.5428882352941177,
      0.379162109375,
      0.5511058823529412,

In [31]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 1) Må lage en liste med alle pose-verdiene
# 2) Scale denne listen
# 3) Erstatte originale pose-verdier med scalete verdier

for p in data['data']:                # For hver frame_index
    for skel in p['skeleton']:        # For hver
        for value in skel['pose']:
            #print(scaler.fit(value))  
            print(str(value))
        for value in skel['score']:
            #print(scaler.fit(value))
            print(str(value))
            

0
0
202.65
110.964
191.534
114.435
174.85
113.715
165.154
97.071
213.052
103.307
190.151
75.5128
185.268
54.6743
173.481
160.268
141.511
211.675
143.605
254.079
185.276
163.756
160.27
210.294
163.739
254.076
0
0
0
0
0
0
201.941
81.771
0
0.567321
0.418317
0.369593
0.253446
0.429515
0.680225
0.661074
0.44076
0.537363
0.0906767
0.442229
0.532781
0.122772
0
0
0
0.348733
0
0
202.655
110.952
191.542
114.432
174.848
113.724
165.153
97.075
213.055
102.623
190.147
75.5091
185.264
54.0149
174.164
159.592
141.515
210.997
143.607
254.079
185.277
163.753
160.266
210.294
163.74
254.076
0
0
0
0
0
0
201.939
81.7728
0
0.574004
0.424646
0.400929
0.271035
0.438216
0.67724
0.662941
0.441086
0.53699
0.0882459
0.446413
0.536633
0.121232
0
0
0
0.353801
0
0
199.864
113.019
186.664
114.42
174.156
109.56
167.917
97.7517
213.072
99.1464
190.822
74.1339
185.262
53.2938
170.02
158.204
141.495
207.501
147.751
254.078
181.825
163.047
160.288
204.046
163.049
254.075
0
0
0
0
0
0
0
0
0
0.4869
0.370766
0.136064
0.10165


0
0
0.159231
0
174.862
108.188
177.641
135.943
163.754
135.969
154.735
86.665
0
0
190.846
135.264
191.521
87.3458
0
0
174.868
211.681
185.275
254.079
0
0
159.586
208.911
0
0
0
0
171.395
104.009
180.395
104.7
165.162
107.48
185.988
108.851
0.450468
0.486858
0.288949
0.344215
0
0.410613
0.278599
0
0.367961
0.0650399
0
0.346167
0
0
0.410854
0.500066
0.0608725
0.320451
174.861
108.188
177.642
135.943
163.756
135.97
154.735
86.6664
0
0
190.846
135.262
191.521
87.349
0
0
174.871
211.678
185.277
254.079
0
0
159.585
208.912
0
0
0
0
171.394
104.011
180.394
104.701
165.161
107.483
185.987
108.852
0.449257
0.485357
0.283575
0.332695
0
0.406663
0.27319
0
0.37087
0.0658572
0
0.347607
0
0
0.410182
0.497724
0.0607446
0.318171
174.894
114.443
176.957
135.949
164.442
135.954
156.813
86.6471
170.027
58.8791
190.848
135.264
191.541
84.5648
184.598
57.4683
164.441
210.98
0
0
0
0
174.176
209.607
0
0
0
0
170.716
110.262
180.441
110.274
163.768
113.712
188.061
110.945
0.847102
0.574544
0.478084
0.695045
0.68

0
181.805
115.136
0.81196
0.564131
0.453687
0
0
0.53352
0.716191
0.691282
0.197961
0
0
0.201649
0
0
0.744593
0.799143
0
0.63085
167.914
120.702
176.269
142.224
165.159
147.08
0
0
0
0
188.058
136.664
189.451
87.3402
185.275
56.0621
169.318
219.32
0
0
0
0
176.246
218.623
0
0
0
0
163.721
118.575
172.086
115.82
0
0
181.148
115.129
0.813795
0.563777
0.454741
0
0
0.525452
0.713561
0.691679
0.191057
0
0
0.197254
0
0
0.738353
0.798071
0
0.629582
167.903
120.704
176.248
142.225
165.14
147.069
0
0
0
0
186.698
136.656
189.44
86.6632
185.279
56.0696
167.92
218.624
0
0
0
0
174.867
217.929
0
0
0
0
163.714
118.572
172.078
115.818
0
0
181.136
115.124
0.805169
0.533486
0.433901
0
0
0.520842
0.715021
0.686519
0.168007
0
0
0.167503
0
0
0.733838
0.796436
0
0.608383
167.223
121.369
175.559
142.9
164.451
147.764
0
0
0
0
186.678
136.65
188.759
86.6506
185.277
56.0745
165.836
215.851
0
0
0
0
175.56
215.16
0
0
0
0
163.063
118.57
172.076
115.831
0
0
181.137
115.827
0.788272
0.533895
0.420925
0
0
0.530612
0.7114

74.8181
185.296
52.6296
169.296
151.247
138.751
190.845
149.168
254.067
181.108
157.491
167.915
201.262
156.109
254.066
0
0
197.77
69.9506
0
0
201.957
72.0483
0.250715
0.495369
0.435205
0.0797844
0
0.586729
0.752052
0.858566
0.34669
0.319714
0.0838479
0.337236
0.167872
0.180526
0
0.362407
0
0.623111
195.696
63.7035
198.501
90.7953
190.149
94.9748
163.063
90.1025
169.973
62.3236
207.52
81.0778
190.845
69.9544
185.94
52.6173
170.001
140.133
137.36
177.649
147.083
241.556
181.136
145.688
164.436
192.921
155.415
247.105
0
0
197.082
60.2448
0
0
201.249
64.3927
0.227123
0.503438
0.436477
0.714985
0.685654
0.6852
0.716077
0.886659
0.414695
0.364765
0.113683
0.393426
0.195512
0.192924
0
0.319481
0
0.555384
195.695
63.7043
199.867
89.4205
191.527
93.59
163.059
90.1109
169.971
62.3259
207.516
81.0723
191.503
69.9625
185.94
52.6209
170.012
140.123
137.353
177.643
147.768
242.94
181.8
145.679
164.45
192.236
155.415
247.113
0
0
197.079
60.2462
0
0
201.252
64.3923
0.229518
0.492003
0.427193
0.70387


108.152
181.813
108.143
165.152
109.555
191.552
108.853
0.834785
0.625227
0.514497
0.790238
0.750351
0.556517
0
0
0.253725
0
0
0.216949
0
0
0.762412
0.865312
0.0641714
0.540937
0
0
0
0
0
0
0
0
0
0
193.623
108.173
192.263
83.1712
185.954
58.136
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0681889
0.786664
0.796656
0
0
0
0
0
0
0
0
0
0
174.865
113.723
181.816
136.646
169.325
138.742
159.597
91.4924
169.991
61.6205
195.7
134.569
192.263
83.1637
185.953
58.1429
174.859
210.981
0
0
0
0
177.641
210.297
0
0
0
0
170.692
109.554
180.42
108.844
0
0
190.13
108.878
0.874098
0.578892
0.455597
0.737773
0.729014
0.502618
0.791856
0.780901
0.238326
0
0
0.202486
0
0
0.654609
0.954894
0
0.393403
174.167
113.728
181.127
136.664
168.611
138.731
159.598
92.1832
169.991
61.6249
196.39
135.247
192.919
84.5566
185.954
58.1416
170.706
210.98
0
0
0
0
181.117
210.287
0
0
0
0
170.021
108.885
180.404
108.854
0
0
190.153
109.573
0.873981
0.643872
0.470879
0.749739
0.73461
0.53355
0.803384
0.804509
0.243131
0


0
0
170.005
220.015
0
0
0
0
160.973
117.903
169.3
117.211
0
0
179.031
117.909
0.869534
0.526884
0.445251
0.0728048
0.0599324
0.518739
0.773879
0.687708
0.211049
0
0
0.165531
0
0
0.785863
0.85122
0
0.760163
164.414
120.703
172.088
143.614
159.576
147.794
153.334
183.893
154.018
210.293
185.29
137.356
186.702
86.6208
185.259
57.4619
157.495
220.716
0
0
0
0
170.01
220.71
0
0
0
0
160.972
117.904
169.3
117.209
0
0
179.032
117.9
0.867266
0.528287
0.44881
0.0735593
0.058938
0.531106
0.776897
0.692845
0.210621
0
0
0.172079
0
0
0.784087
0.85288
0
0.76481
163.779
120.676
172.092
142.924
159.592
147.773
154.02
183.203
0
0
185.285
136.674
186.704
86.6285
185.26
57.4578
158.185
218.632
0
0
0
0
170.704
217.937
0
0
0
0
160.292
117.211
169.301
116.502
0
0
179.016
117.888
0.878807
0.517628
0.441493
0.0736551
0
0.524568
0.772019
0.705175
0.199853
0
0
0.165492
0
0
0.773207
0.845329
0
0.743622
163.767
120.671
172.772
142.914
160.266
147.767
154.032
182.514
0
0
185.955
136.663
186.699
86.6188
185.26
57.457

0
0
0
0
205.426
92.8951
192.266
78.9957
185.972
54.6911
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.276258
0.623133
0.666881
0
0
0
0
0
0
0
0
0
0
0
0
190.142
108.176
179.021
107.471
158.2
90.8077
170.01
60.2328
201.261
102.621
0
0
0
0
147.086
179.722
152.644
247.802
0
0
175.561
172.78
0
0
0
0
0
0
0
0
0
0
0
0
0
0.368531
0.296224
0.720122
0.528524
0.113113
0
0
0.196932
0.113786
0
0.078723
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
203.326
86.6188
192.231
75.5524
185.96
53.9979
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.16921
0.316522
0.454989
0
0
0
0
0
0
0
0
0
0
0
0
187.376
104.712
201.947
101.226
198.488
76.9267
186.001
53.9822
0
0
0
0
0
0
159.589
142.22
135.97
169.315
147.082
247.794
178.331
149.162
168.6
183.89
153.313
239.475
0
0
0
0
0
0
0
0
0
0.407803
0.0981725
0.407701
0.363766
0
0
0
0.274859
0.266708
0.165186
0.162093
0.12643
0.20334
0
0
0
0
181.117
87.3325
188.063
103.326
197.798
89.4343
198.481
76.9177
185.986
53.3287
191.537
108.186
0
0
0
0
161.662
141.533
135.96
169.988
1

In [12]:
type(data)

dict

In [3]:
help(MinMaxScaler)

Help on class MinMaxScaler in module sklearn.preprocessing.data:

class MinMaxScaler(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin)
 |  Transforms features by scaling each feature to a given range.
 |  
 |  This estimator scales and translates each feature individually such
 |  that it is in the given range on the training set, i.e. between
 |  zero and one.
 |  
 |  The transformation is given by::
 |  
 |      X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
 |      X_scaled = X_std * (max - min) + min
 |  
 |  where min, max = feature_range.
 |  
 |  This transformation is often used as an alternative to zero mean,
 |  unit variance scaling.
 |  
 |  Read more in the :ref:`User Guide <preprocessing_scaler>`.
 |  
 |  Parameters
 |  ----------
 |  feature_range : tuple (min, max), default=(0, 1)
 |      Desired range of transformed data.
 |  
 |  copy : boolean, optional, default True
 |      Set to False to perform inplace row normalization and avoid a
 |   